# Assignment 2, Problem 9 Computation Teamwork
## Group: Austin Wang, Lawrence Moore, Joe Higgins


In [12]:
# Import necessary modules
import cvxpy as cvx
import numpy as np

## Introduction
In this notebook, we will be solving the two-dimensional sensor localization problem with one sensor and three anchors. We assume that we know each of the three distances between the sensor and the anchors. Our goal is to use this information to determine the location of the sensor.

We will formulate this problem as both an SOCP relaxation problem and an SDP relaxation problem to analyze how the two methods compare.

# Results summary

## SOCP

### Case 1: Both in Convex Hulls
Optimizer finds both sensors succesfully in this case. 
Ex: s1=[0, 0.1], s2=[0, 0.5]
### Case 2: One in, one out, both in convex hull of anchors
Optimizer finds the sensor in the convex hull correctly, but fails to find the sensor outside of the hull.
Ex: s1=[0, 0.1], s2=[-.15, 1.8]
### Case 3: One in, one out, one not in convex hull of anchors
Optimizer does not find either of the sensors successfully
Ex: s1=[0, 0.1], s2=[-.5, 10]
### Case 4: Neither in convex hull of each other
Optimizer does not find either of the sensors successfully
Ex: s1=[ -.5, .5], s2=[.5, 0.5]

## SDP
### Case 1: Both in Convex Hulls
Optimizer finds both sensors succesfully in this case. 
### Case 2: One in, one out, both in convex hull of anchors
Optimizer finds both sensors succesfully in this case. 
### Case 3: One in, one out, one not in convex hull of anchors
Optimizer does not find either of the sensors successfully
### Case 4: Neither in convex hull of each other
Optimizer does not find either of the sensors successfully

## Problem Data

In [8]:
# Define anchors
anchors = np.matrix([
    [ 1, 0],
    [-1, 0],
    [ 0, 1.4]
])

# Number of dimensions
n = 2

# Sensor locations
# sensor_location_in = [0.44, 0.25] # Sensor inside convex hull
# sensor_location_out = [1, 1]      # Sensor outside convex hull

# case 1: sensor_location_both_in
s1 = np.matrix([
    [ 0, .1],
    [0, 0.5]]) 

# case 2: sensor_location_one_in_one_out_in_anchors
s1 = np.matrix([
    [ 0, .1],
    [-.15, 1.8]])

# case 3: sensor_location_one_in_one_out_one_not_in_anchors
s3 = np.matrix([
    [ 0, .1],
    [-.5, 10]])

# case 4: one in one out, not in convex hull of anchors
s3 = np.matrix([
    [ 0, .1],
    [0.05, 10]])


# case 4: not in each others' convex hull
s4 = np.matrix([
    [ -.5, .5],
    [.5, 0.5]])

# Distances from anchors

d0 = list(map(lambda a: np.linalg.norm(s1[0, :] - a), anchors))
d1 = list(map(lambda a: np.linalg.norm(s1[1, :] - a), anchors))
d_both_sensor = np.linalg.norm(s1[0, :] - s1[1, :])
# d_out = list(map(lambda a: np.linalg.norm(sensor_location_out - a), anchors))

## Part 1: SOCP Relaxation Problem Experimentation

We formulate the SOCP relaxation problem below for a sensor in the convex hull:

In [13]:
# Construct the problem.
x = cvx.Variable(2,n)
objective = cvx.Minimize(0)

x1 = x[0,:]
x2 = x[1, :]

constraints = [cvx.norm(x1 - anchors[0]) <= d0[0],
               cvx.norm(x1 - anchors[1]) <= d0[1],
               cvx.norm(x1 - x2) <= d_both_sensor,
               cvx.norm(x2 - anchors[1]) <= d1[1],
               cvx.norm(x2 - anchors[2]) <= d1[2]
              ]

prob = cvx.Problem(objective, constraints)

Solving the problem yields the following:

In [14]:
# The optimal objective is returned by prob.solve().
result = prob.solve(solver = 'MOSEK')

# The optimal value for x is stored in x.value.
print('Results for Sensor Inside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(sensor_location_both_in))
print('SOCP optimal sensor location 1: {}'.format(x1.value))
print('SOCP optimal sensor location 2: {}'.format(x2.value))

Results for Sensor Inside of Convex Hull of Anchors
---------------------------------------------------


NameError: name 'sensor_location_both_in' is not defined

As one can see, the SOCP relaxation problem solution yields the correct sensor location for a sensor in the convex hull of the anchors.

Now let us see what happens if the sensor is outside of the convex hull:

In [11]:
# Update constraints for sensor outside of the convex hull
constraints = [cvx.norm(x - anchors[0]) <= d_out[0],
               cvx.norm(x - anchors[1]) <= d_out[1],
               cvx.norm(x - anchors[2]) <= d_out[2]]

prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

print('Results for Sensor Outside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(sensor_location_out))
print('SOCP optimal sensor location : {}'.format(x.value))

NameError: name 'x' is not defined

Interestingly enough, the SOCP relaxation problem solution does not yield the correct sensor location for a sensor outside of the convex hull of the anchors. Let us see if the SDP relaxation problem differs.

## Part 2: SDP Relaxation Problem Experimentation

The following functions will be useful in the code that follows:

In [4]:
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

def col_vec_4elem(a,b,c,d):
    return np.matrix([[a],[b],[c],[d]])

We formulate the SDP relaxation problem below for a sensor in the convex hull:

In [9]:
# Constraints will define the proper structure for Z. The optimal value will be 
# located in the top right two elements of Z.
Z = cvx.Semidef(4)

# Objective does not matter; 
# We are simply concerned with solving feasibility conditions
objective = cvx.Minimize(0) 


v0 = col_vec_4elem(1,0,0,0)
v1 = col_vec_4elem(0,1,0,0)
v2 = col_vec_4elem(1,1,0,0)

a0 = col_vec_4elem(anchors[0,0], anchors[0,1], -1, 0)
a1 = col_vec_4elem(anchors[1,0], anchors[1,1], -1, 0)

a2 = col_vec_4elem(anchors[1,0], anchors[1,1], 0, -1)
a3 = col_vec_4elem(anchors[2,0], anchors[2,1], 0, -1)

v3 = col_vec_4elem(0, 0, 1, -1)

constraints = [
    sum_elem_product(v0*np.transpose(v0), Z) == 1,
    sum_elem_product(v1*np.transpose(v1), Z) == 1,
    sum_elem_product(v2*np.transpose(v2), Z) == 2,
    sum_elem_product(a0*np.transpose(a0), Z) == cvx.square(d0[0]),
    sum_elem_product(a1*np.transpose(a1), Z) == cvx.square(d0[1]),
    sum_elem_product(a2*np.transpose(a2), Z) == cvx.square(d1[1]),
    sum_elem_product(a3*np.transpose(a3), Z) == cvx.square(d1[2]),
    sum_elem_product(v3*np.transpose(v3), Z) == cvx.square(d_both_sensor)
]

Solving the problem and extracting the solution from the top right two elements of Z yields:

In [10]:
prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

# Optimal solution
x_1_star = [Z[0,2].value, Z[1,2].value]
x_2_star = [Z[0,3].value, Z[1,3].value]

print('Results for Sensor Inside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(s1))
print('SDP optimal sensor 1 location : {}'.format(x_1_star))
print('SDP optimal sensor 2 location : {}'.format(x_2_star))

Results for Sensor Inside of Convex Hull of Anchors
---------------------------------------------------
True Sensor Location: [[ 1.   0.7]
 [ 0.  -0.3]]
SDP optimal sensor 1 location : [1.0, -0.047102782992636635]
SDP optimal sensor 2 location : [-0.20474325496134232, -0.15375481788475545]


Like in the SOCP example, the SDP relaxation problem solution yields the correct location for the sensor located inside of the convex hull of the anchors.

Now we test SDP for the sensor outside of the convex hull:

In [9]:
# Update constraints for sensor outside of the convex hull
constraints = [
    sum_elem_product(v0*np.transpose(v0), Z) == 1,
    sum_elem_product(v1*np.transpose(v1), Z) == 1,
    sum_elem_product(v2*np.transpose(v2), Z) == 2,
    sum_elem_product(a0*np.transpose(a0), Z) == cvx.square(d_out[0]),
    sum_elem_product(a1*np.transpose(a1), Z) == cvx.square(d_out[1]),
    sum_elem_product(a2*np.transpose(a2), Z) == cvx.square(d_out[2])
]

prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

# Optimal solution
x_star = [Z[0,2].value, Z[1,2].value]

print('Results for Sensor Outside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(sensor_location_out))
print('SDP optimal sensor location : {}'.format(x_star))

Results for Sensor Outside of Convex Hull of Anchors
---------------------------------------------------
True Sensor Location: [1, 1]
SDP optimal sensor location : [1.0000000000000002, 1.0]


The SDP relaxation problem solution does yield the correct location for the sensor located outside of the convex hull. Therefore, we conclude that the SDP formulation is preferable to the SOCP formulation in solving our particular sensor localization problem if we care to generalize the possible locations for the sensor.

As a final test of the SDP formulation, we consider choosing a sensor that is very far outside the convex hull of the anchors:

In [10]:
sensor_location_out2 = [1000, 1000]
d_out2 = list(map(lambda a: np.linalg.norm(sensor_location_out2 - a), anchors))

# Update constraints for sensor very far outside of the convex hull
constraints = [
    sum_elem_product(v0*np.transpose(v0), Z) == 1,
    sum_elem_product(v1*np.transpose(v1), Z) == 1,
    sum_elem_product(v2*np.transpose(v2), Z) == 2,
    sum_elem_product(a0*np.transpose(a0), Z) == cvx.square(d_out2[0]),
    sum_elem_product(a1*np.transpose(a1), Z) == cvx.square(d_out2[1]),
    sum_elem_product(a2*np.transpose(a2), Z) == cvx.square(d_out2[2])
]

prob = cvx.Problem(objective, constraints)
result = prob.solve(solver = 'MOSEK')

# Optimal solution
x_star = [Z[0,2].value, Z[1,2].value]

print('Results for Sensor Outside of Convex Hull of Anchors')
print('---------------------------------------------------')
print('True Sensor Location: {}'.format(sensor_location_out2))
print('SDP optimal sensor location : {}'.format(x_star))

Results for Sensor Outside of Convex Hull of Anchors
---------------------------------------------------
True Sensor Location: [1000, 1000]
SDP optimal sensor location : [1000.0, 999.9999999999418]


As one can see, SDP is even able to find the correct sensor location for a point very far outside of the convex hull of the anchors. Therefore, we conclude that SDP is likely able to find the exact correct solution for every sensor location on the plane.